<a href="https://colab.research.google.com/github/christinium/Health/blob/main/echo_note_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Echo Note Training with Gemma LLM
This notebook demonstrates how to fine-tune the Gemma language model on echocardiogram notes. The model will learn to analyze echo reports and provide structured assessments across 19 different cardiac features.

## Overview
1. Set up Google Drive and dependencies
2. Load and prepare echo note data
3. Configure data labels and formatting
4. Clean and preprocess the dataset
5. Create train/tune/test splits
6. Convert to HuggingFace format
7. Configure and train the model
8. Evaluate results

## 1. Set Up Google Drive
First, let's mount Google Drive to access our data and save model checkpoints. This step is essential for persisting our data and model files across Colab sessions.

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up working directory
import os
WORKING_DIR = '/content/drive/MyDrive/echo_training/'  # Change this to your preferred location
os.makedirs(WORKING_DIR, exist_ok=True)
os.chdir(WORKING_DIR)

print(f"Working directory set to: {WORKING_DIR}")
print("Contents:", os.listdir())

Mounted at /content/drive
Working directory set to: /content/drive/MyDrive/echo_training/
Contents: ['echo_dataset.csv', 'echo_train.csv', 'echo_tune.csv', 'echo_test.csv', 'gemma_echo_finetuned']


## 2. Install Required Dependencies
Install and import all necessary libraries for the project. We'll need:
- transformers: For the Gemma model and training
- datasets: For data handling
- pandas & numpy: For data manipulation
- sklearn: For data splitting
- torch: For deep learning operations

In [2]:
# Install required packages
!pip install transformers datasets torch numpy pandas scikit-learn -q

# Import libraries
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import ast
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, DataCollatorForLanguageModeling, TrainingArguments

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4


## 3. Load and Prepare Echo Data
Load your echo data from the CSV file. Make sure your data file is uploaded to the working directory in Google Drive.

In [3]:
# Load your CSV file
# Change 'your_echo_data.csv' to your actual filename in Google Drive
df = pd.read_csv('/content/drive/MyDrive/echo_training/echo_dataset.csv')

print("Data loaded successfully!")
print(f"Total samples: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst row:")
print(df.head(1))

Data loaded successfully!
Total samples: 45794
Columns: ['text', 'labels']

First row:
                                                text  \
0  PATIENT/TEST INFORMATION:\nIndication: Left ve...   

                                              labels  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, -3, -3, -3, 0, 0, ...  


## 4. Configure Data Labels
Set up the 19 cardiac feature labels and create functions to parse the label data. These labels represent different aspects of cardiac function that we'll be predicting.

In [4]:
# Define label names
LABEL_NAMES = [
    'LA_cavity', 'RA_dilated', 'LV_systolic', 'LV_cavity',
    'LV_wall', 'RV_cavity', 'RV_systolic', 'AV_stenosis',
    'MV_stenosis', 'TV_regurgitation', 'TV_stenosis',
    'TV_pulm_htn', 'AV_regurgitation', 'MV_regurgitation',
    'RA_pressure', 'LV_diastolic', 'RV_volume_overload',
    'RV_wall', 'RV_pressure_overload'
]

def parse_labels(label_str):
    """Convert string representation of list to actual list."""
    if isinstance(label_str, str):
        return ast.literal_eval(label_str)
    elif isinstance(label_str, list):
        return label_str
    else:
        return label_str

# Parse labels
df['labels_parsed'] = df['labels'].apply(parse_labels)

# Verify labels
assert all(len(labels) == 19 for labels in df['labels_parsed']), \
    "Not all label arrays have 19 values!"

print("\nLabels parsed successfully!")
print(f"Example labels: {df['labels_parsed'].iloc[0]}")

# Display label distribution
label_counts = pd.DataFrame([
    [label, sum(x[i] for x in df['labels_parsed'])]
    for i, label in enumerate(LABEL_NAMES)
], columns=['Label', 'Count'])

print("\nLabel Distribution:")
print(label_counts)


Labels parsed successfully!
Example labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, -3, -3, -3, 0, 0, 0, 0, 0, 0, 0]

Label Distribution:
                   Label  Count
0              LA_cavity -22504
1             RA_dilated  13251
2            LV_systolic   8901
3              LV_cavity   2829
4                LV_wall  12501
5              RV_cavity -16274
6            RV_systolic -18107
7            AV_stenosis   7572
8            MV_stenosis    261
9       TV_regurgitation -22032
10           TV_stenosis  -6320
11           TV_pulm_htn -12676
12      AV_regurgitation   6303
13      MV_regurgitation -30278
14           RA_pressure    921
15          LV_diastolic  -5736
16    RV_volume_overload  -3504
17               RV_wall  -4190
18  RV_pressure_overload  -3259


In [5]:
df.head()

,text,labels,labels_parsed
0,PATIENT/TEST INFORMATION:\nIndication: Left ve...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, -3, -3, -3, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, -3, -3, -3, 0, 0, ..."
1,PATIENT/TEST INFORMATION:\nIndication: Endocar...,"[1, 0, 0, 0, 1, 0, -3, 0, 0, 0, 0, -3, 0, 1, 0...","[1, 0, 0, 0, 1, 0, -3, 0, 0, 0, 0, -3, 0, 1, 0..."
2,PATIENT/TEST INFORMATION:\nIndication: Left ve...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ..."
3,PATIENT/TEST INFORMATION:\nIndication: Endocar...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 3, 1, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 3, 1, 0, ..."
4,PATIENT/TEST INFORMATION:\nIndication: Left ve...,"[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 5. Format Data for Training
Create a formatting function to prepare the data for Gemma. We'll structure the input as a prompt with the echo report and expected output format.

In [6]:
def format_echo_prompt(row):
    """Format echo report into Gemma instruction format."""
    input_text = row['text']  # Change 'input' to your actual column name
    labels = row['labels_parsed']

    # Create formatted label string
    label_pairs = [f"{LABEL_NAMES[i]}: {labels[i]}" for i in range(19)]
    label_text = "\n".join(label_pairs)

    prompt = f"""<start_of_turn>user
Analyze this echocardiogram report and provide assessment values for each cardiac feature. Output should be in the format "feature: value" for each of the 19 features.

Report:
{input_text}<end_of_turn>
<start_of_turn>model
{label_text}<end_of_turn>"""

    return prompt

# Apply formatting
df['text'] = df.apply(format_echo_prompt, axis=1)

print("FORMATTED EXAMPLE:")
print("="*70)
print(df['text'].iloc[0])
print("="*70)

FORMATTED EXAMPLE:
<start_of_turn>user
Analyze this echocardiogram report and provide assessment values for each cardiac feature. Output should be in the format "feature: value" for each of the 19 features.

Report:
PATIENT/TEST INFORMATION:
Indication: Left ventricular function. Heart for transplant.
Height: (in) 64
Weight (lb): 234
BSA (m2): 2.09 m2
BP (mm Hg): 123/50
HR (bpm): 102
Status: Inpatient
Date/Time: [**2178-6-8**] at 10:19
Test: Portable TTE(Focused views)
Doppler: Focused pulse and color flow
Contrast: None
Technical Quality: Adequate


INTERPRETATION:

Findings:

LEFT ATRIUM: The left atrium is normal in size.

RIGHT ATRIUM/INTERATRIAL SEPTUM: The right atrium is normal in size.

LEFT VENTRICLE: Left ventricular wall thickness, cavity size, and systolic
function are normal (LVEF>55%). Regional left ventricular wall motion is
normal.

RIGHT VENTRICLE: Right ventricular chamber size and free wall motion are
normal.

AORTA: The aortic root is normal in diameter.

AORTIC VAL

## 6. Data Cleaning and Preprocessing
Clean the dataset by removing duplicates, handling missing values, and filtering by text length if needed.

In [7]:
print(f"Before cleaning: {len(df)} samples")

# Remove any rows with missing data
df = df.dropna(subset=['text'])

# Remove duplicates based on input text
df = df.drop_duplicates(subset=['text'])

# Analyze text length
df['text_length'] = df['text'].str.len()
print(f"\nText length stats:")
print(df['text_length'].describe())

# Optional: Remove extremely short or long examples
# Uncomment and adjust thresholds as needed
# df = df[(df['text_length'] > 100) & (df['text_length'] < 4096)]

df = df.drop(columns=['text_length'])

print(f"After cleaning: {len(df)} samples")

Before cleaning: 45794 samples

Text length stats:
count    45794.000000
mean      2867.589466
std        736.378007
min        828.000000
25%       2460.250000
50%       2899.000000
75%       3319.000000
max       6696.000000
Name: text_length, dtype: float64
After cleaning: 45794 samples


## 7. Create Dataset Splits
Split the data into training (70%), tuning/validation (15%), and test (15%) sets. We'll save these splits to CSV files in Google Drive for reproducibility.

In [8]:
# First split: separate test set (15%)
train_tune_df, test_df = train_test_split(
    df,
    test_size=0.15,
    random_state=42,
    shuffle=True
)

# Second split: separate train and tune from remaining 85%
train_df, tune_df = train_test_split(
    train_tune_df,
    test_size=0.1765,  # This gives us 15% of original
    random_state=42,
    shuffle=True
)

print("DATASET SPLITS:")
print("="*70)
print(f"Training set:   {len(train_df)} samples ({len(train_df)/len(df)*100:.1f}%)")
print(f"Tuning set:     {len(tune_df)} samples ({len(tune_df)/len(df)*100:.1f}%)")
print(f"Test set:       {len(test_df)} samples ({len(test_df)/len(df)*100:.1f}%)")
print(f"Total:          {len(df)} samples")

# Save splits to CSV
train_df.to_csv('echo_train.csv', index=False)
tune_df.to_csv('echo_tune.csv', index=False)
test_df.to_csv('echo_test.csv', index=False)

print("\n✓ Splits saved to CSV files in Google Drive:")

DATASET SPLITS:
Training set:   32053 samples (70.0%)
Tuning set:     6871 samples (15.0%)
Test set:       6870 samples (15.0%)
Total:          45794 samples

✓ Splits saved to CSV files in Google Drive:


## 8. Convert to HuggingFace Format
Transform our data into HuggingFace datasets for efficient training. This includes creating a DatasetDict and tokenizing the data.

In [9]:
# Create datasets from the DataFrames
train_dataset = Dataset.from_pandas(train_df[['text']], preserve_index=False)
tune_dataset = Dataset.from_pandas(tune_df[['text']], preserve_index=False)
test_dataset = Dataset.from_pandas(test_df[['text']], preserve_index=False)

# Combine into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': tune_dataset,
    'test': test_dataset
})

print("HUGGING FACE DATASET:")
print("="*70)
print(dataset)

HUGGING FACE DATASET:
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 6871
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6870
    })
})


## 9. Configure Model and Tokenizer
Set up the Gemma model and tokenizer. We'll use the 2B parameter version for prototyping, but you can switch to 9B for production.

In [10]:
# Load model and tokenizer
model_name = 'google/gemma-2b'  # or 'google/gemma-9b' for production
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    """Tokenize the text data."""
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,  # Longer for medical reports
        padding="max_length",
    )

# Tokenize all splits
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing datasets",
)

print("TOKENIZED DATASETS:")
print("="*70)
print(tokenized_datasets)

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Tokenizing datasets:   0%|          | 0/32053 [00:00<?, ? examples/s]

Tokenizing datasets:   0%|          | 0/6871 [00:00<?, ? examples/s]

Tokenizing datasets:   0%|          | 0/6870 [00:00<?, ? examples/s]

TOKENIZED DATASETS:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6871
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6870
    })
})


## 10. Configure Training
Set up the training arguments and create the trainer. We'll use settings optimized for medical domain training.

In [11]:
# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments optimized for medical domain
training_args = TrainingArguments(
    output_dir="./gemma_echo_finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=False,
    bf16=True,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    warmup_steps=100,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    max_grad_norm=1.0,
    report_to="none",
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

print("\n✓ Trainer configured for echo report data!")
print(f"Training on:   {len(tokenized_datasets['train'])} samples")
print(f"Validating on: {len(tokenized_datasets['validation'])} samples")
print(f"Test set:      {len(tokenized_datasets['test'])} samples (held out)")


✓ Trainer configured for echo report data!
Training on:   32053 samples
Validating on: 6871 samples
Test set:      6870 samples (held out)


## 11. Model Training and Evaluation
Train the model and evaluate its performance. We'll also include a helper function for final evaluation on the test set.

In [13]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 42.7 MB/s eta 0:00:00


In [14]:
# Training
print("Starting training...")
trainer.train()

Starting training...


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 2688 has 14.73 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 6.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:


def evaluate_on_test_set():
    """Evaluate the fine-tuned model on the held-out test set."""
    print("\nEVALUATING ON TEST SET")
    print("="*70)

    test_results = trainer.evaluate(tokenized_datasets['test'])

    print("\nTest Set Results:")
    for key, value in test_results.items():
        print(f"  {key}: {value:.4f}")

    return test_results

# Evaluate on test set
test_results = evaluate_on_test_set()

# Save the final model
output_dir = "final_model"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("\n✓ Model and tokenizer saved to:", output_dir)

## Important Notes for Medical Data

1. **Data Balance**
   - Check if your 19 labels are balanced
   - Medical data often has class imbalance
   - Consider stratified splitting if certain conditions are rare

2. **Evaluation Metrics**
   - Loss alone may not be sufficient
   - Consider implementing custom metrics (F1, precision, recall per label)
   - Medical predictions need high precision

3. **Model Size**
   - Gemma 2B is good for prototyping
   - Consider Gemma 9B for production if accuracy is critical
   
4. **Training Tips**
   - Monitor validation loss closely
   - Stop if validation loss stops decreasing
   - Medical domain may need more epochs (5-10)
   - Lower learning rate is safer for specialized domains

5. **Prompt Engineering**
   - Current format outputs all 19 values at once
   - Consider one value at a time for higher reliability
   - Add medical context in prompts if needed

Remember to validate the model thoroughly before any clinical use!